In [3]:
import os
os.getcwd()

'/home/rajan/Desktop/thesis/thesis_work/minimalist_diffusion'

In [4]:
epoch_ = 0
dataset_path = '/home/rajan/Desktop/thesis/thesis_work'
model_preloading_path = '/home/rajan/Desktop/thesis/thesis_work/minimalist_diffusion_ckpts/mode_28_ckpt.pth'
model_checkpoint_path = f'/home/rajan/Desktop/thesis/thesis_work/minimalist_drl_ckpts/model_{epoch_}_ckpt.pth'

In [5]:
from minimalist_diff_rl_model import minimalDiffRl
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch
from torch.optim import Adam
from collections import OrderedDict
import torchvision.transforms as transforms
import numpy as np


In [4]:

#Setting up the parameters
device  = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 128
last_epoch = 0
num_epochs = 100
learning_rate = 1e-4
model = torch.nn.DataParallel(minimalDiffRl())
model = model.to(device = device)
model_last_trained_path = f'/home/rajan/Desktop/thesis/thesis_work/minimalist_drl_ckpts/model_{last_epoch}_ckpt.pth'
#load the pretrained  DiffusionNet from the checkpoint --> Note that models are being saved in the form of Datparallel objects
#so --> need to bring in a new dictionary -> 
#approach taken from : https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/4

new_state_dict = OrderedDict()

"""loading parameters for the diffusion net"""

# ckpt = torch.load(model_preloading_path)
# for k, v in ckpt.items():
#     name = k[7:] # remove module.
#     new_state_dict[name] = v

# model.module.encoder_drl.load_state_dict(new_state_dict)


"""loading parameters for continual learning"""
# ckpt_last = torch.load(model_last_trained_path)
# model.load_state_dict(ckpt_last)

    
optimizer = Adam(model.parameters(), learning_rate)

#defining the loss function 
def loss_fn(model: minimalDiffRl, x1:torch.tensor, alpha_tensor:torch.tensor):
    """model : minimalDiffRl object wtih alpha embeddings
       x     : input image
    """
    x0 = torch.randn_like(x1).to(device)
    alpha = torch.randn(batch_size,).uniform_(0,1).to(device)
    blended_x  = (1.-alpha.view(-1,1,1,1))*x0 + alpha.view(-1,1,1,1)*x1
    model_output = model(blended_x, alpha)

    loss = torch.mean(torch.square(model_output - (x1-x0)))
    return loss


datasets = MNIST(dataset_path,train=True,transform=transforms.ToTensor(),download=False)
data_loader = DataLoader(dataset=datasets,batch_size=batch_size,shuffle=True,num_workers=4)
t = torch.ones(batch_size).to(device=device)
#Training loop 
for epoch in range(last_epoch, last_epoch+num_epochs):
    avg_loss = 0
    num_items = 0
    for i , (x, y) in enumerate(data_loader):
        if x.shape[0] != batch_size:
            continue
        x = x.to(device)
        x  = x*2.0 - 1
        loss = loss_fn(model,x,t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        num_items += x.shape[0]
        avg_loss += loss.item() * x.shape[0]
    print(f'Epoch number : {epoch}')
    print('Average Loss: {:5f}'.format(avg_loss / num_items))
  # Update the checkpoint after each epoch of training.
    torch.save(model.state_dict(),f'/home/rajan/Desktop/thesis/thesis_work/minimalist_drl_ckpts/model_{epoch}_ckpt.pth')
    epoch_ += 1


Epoch number : 0
Average Loss: 1.746806
Epoch number : 1
Average Loss: 1.566619
Epoch number : 2
Average Loss: 1.501240
Epoch number : 3
Average Loss: 1.437793
Epoch number : 4
Average Loss: 1.416012
Epoch number : 5
Average Loss: 1.403518
Epoch number : 6
Average Loss: 1.393356
Epoch number : 7
Average Loss: 1.385552
Epoch number : 8
Average Loss: 1.378686
Epoch number : 9
Average Loss: 1.375112
Epoch number : 10
Average Loss: 1.369979
Epoch number : 11
Average Loss: 1.365452
Epoch number : 12
Average Loss: 1.362093
Epoch number : 13
Average Loss: 1.358348
Epoch number : 14
Average Loss: 1.354212
Epoch number : 15
Average Loss: 1.351843
Epoch number : 16
Average Loss: 1.348698
Epoch number : 17
Average Loss: 1.346386
Epoch number : 18
Average Loss: 1.343361
Epoch number : 19
Average Loss: 1.340706
Epoch number : 20
Average Loss: 1.337741
Epoch number : 21
Average Loss: 1.335737
Epoch number : 22
Average Loss: 1.333202
Epoch number : 23
Average Loss: 1.332494
Epoch number : 24
Average 

In [ ]:

from torchvision.utils import make_grid
import numpy as np
import torch


device = 'cuda'

time_steps = 128

#define an alpha schedule 
def alpha_linear_schedule(steps):
    """steps : number of steps
    """
    return torch.linspace(0,1,steps).to(device=device)

def cosine_schedule(alpha_schedule):
    """alpha_schedule : a linear alpha schedule of t/T where T : max non of steps
    """
    return 1. - torch.cos(alpha_schedule * (torch.pi/2.)).to(device=device)
    
#for sampling procedure
alpha_schedule = alpha_linear_schedule(time_steps)

# Need to check the sampling step --> 
#Sampling steps 
def sampler(alpha_schedule,model):

    x_alpha = torch.rand(32,1,28,28).to(device=device)
    t = torch.ones(x_alpha.shape[0]).to(device=device)
    for i  in range(1,len(alpha_schedule)):
        alpha_tensor = t*alpha_schedule[i-1]
        x_alpha = x_alpha + (alpha_schedule[i] - alpha_schedule[i-1])*model(x_alpha,alpha_tensor)

    return x_alpha

def cosine_function(t: int, T: int):
    """ t: unit time
        T: total time steps 
    """
    return 1. - torch.cos(torch.tensor((t/T)* (torch.pi/2))).to(device=device)
 
def given_Sampler(model,nb_step):
    x_alpha = torch.randn(32,1,28,28).to(device=device)
    for t in range(nb_step):
        alpha_start = (t/nb_step)
        alpha_end =((t+1)/nb_step)

        d = model(x_alpha, torch.tensor(alpha_start, device=x_alpha.device))['sample']
        x_alpha = x_alpha + (alpha_end-alpha_start)*d

    return x_alpha 

def improved_sampler(model:minimalDiffRl, steps):
    """model : 
       steps : total time steps T
    """
    x_alpha = torch.randn(32,1,28,28).to(device=device)
    ones = torch.ones(x_alpha.shape[0]).to(device=device)
    for t in range(steps):
        alpha_half_tensor = ones*cosine_function(t+0.5,steps)
        alpha_tensor = ones*cosine_function(t,steps)
        x_alpha_half  =  x_alpha + (cosine_function(t+0.5,steps) - cosine_function(t,steps))* model(x_alpha,alpha_tensor)
        x_alpha       =  x_alpha + (cosine_function(t+1,steps)   - cosine_function(t,steps))* model(x_alpha_half,alpha_half_tensor)  

    return x_alpha

#sampling procedure
model = minimalDiffRl()



new_state_dict = OrderedDict()

"""loading parameters for the diffusion net"""


ckpt = torch.load('/home/rajan/Desktop/thesis/thesis_work/minimalist_drl_ckpts/model_118_ckpt.pth', map_location=device)
for k, v in ckpt.items():
    name = k[7:] # remove module.
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
model.to(device=device)
model.eval()
# samples = sampler(alpha_schedule, model)
# samples = given_Sampler(model, nb_step=128)
samples = improved_sampler(model,steps=time_steps)


samples = samples.clamp(0.0, 1.0)
import matplotlib.pyplot as plt
%matplotlib inline

sample_grid = make_grid(samples, nrow=int(np.sqrt(32)))

plt.figure(figsize=(6,6))
plt.axis('off')
plt.imshow(sample_grid.permute(1,2,0).cpu(), vmin=0., vmax=1.)
plt.show()

## Trying out pixelated Alphas 


In [6]:
from minimal_diff_rl_model2 import minimalDiffRl_revised

In [11]:
device  = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 128
last_epoch = 196
num_epochs = 100
learning_rate = 1e-4
model = torch.nn.DataParallel(minimalDiffRl_revised())
model = model.to(device = device)
model_last_trained_path = f'/home/rajan/Desktop/thesis/thesis_work/minimalist_drl_ckpts/model_{last_epoch}_ckpt.pth'
#load the pretrained  DiffusionNet from the checkpoint --> Note that models are being saved in the form of Datparallel objects
#so --> need to bring in a new dictionary -> 
#approach taken from : https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/4

new_state_dict = OrderedDict()

"""loading parameters for the diffusion net"""

# ckpt = torch.load(model_preloading_path)
# for k, v in ckpt.items():
#     name = k[7:] # remove module.
#     new_state_dict[name] = v

# model.module.encoder_drl.load_state_dict(new_state_dict)


"""loading parameters for continual learning"""
ckpt_last = torch.load(model_last_trained_path)
model.load_state_dict(ckpt_last)

    
optimizer = Adam(model.parameters(), learning_rate)


#defining the loss function 
def loss_fn(model: minimalDiffRl, x1:torch.tensor):
    """model : minimalDiffRl object wtih alpha embeddings
       x     : input image
    """
    x0 = torch.randn_like(x1).to(device)
    alpha = torch.randn_like(x1).uniform_(0,1).to(device)
    blended_x  = (1.-alpha)*x0 + alpha*x1
    model_output = model(blended_x, alpha)

    loss = torch.mean(torch.square(model_output - (x1-x0)))
    return loss


datasets = MNIST(dataset_path,train=True,transform=transforms.ToTensor(),download=False)
data_loader = DataLoader(dataset=datasets,batch_size=batch_size,shuffle=True,num_workers=4)


datasets = MNIST(dataset_path,train=True,transform=transforms.ToTensor(),download=False)
data_loader = DataLoader(dataset=datasets,batch_size=batch_size,shuffle=True,num_workers=4)

#Training loop 
for epoch in range(last_epoch, last_epoch+num_epochs):
    avg_loss = 0
    num_items = 0
    for i , (x, y) in enumerate(data_loader):
        if x.shape[0] != batch_size:
            continue
        x = x.to(device)
        loss = loss_fn(model,x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        num_items += x.shape[0]
        avg_loss += loss.item() * x.shape[0]
    print(f'Epoch number : {epoch}')
    print('Average Loss: {:5f}'.format(avg_loss / num_items))
  # Update the checkpoint after each epoch of training.
    torch.save(model.state_dict(),f'/home/rajan/Desktop/thesis/thesis_work/minimalist_drl_ckpts/model_{epoch}_ckpt.pth')
    epoch_ += 1



Epoch number : 196
Average Loss: 0.405729
Epoch number : 197
Average Loss: 0.404205
Epoch number : 198
Average Loss: 0.404395
Epoch number : 199
Average Loss: 0.404349
Epoch number : 200
Average Loss: 0.404162
Epoch number : 201
Average Loss: 0.404181
Epoch number : 202
Average Loss: 0.404128
Epoch number : 203
Average Loss: 0.403879
Epoch number : 204
Average Loss: 0.403803
Epoch number : 205
Average Loss: 0.403711
Epoch number : 206
Average Loss: 0.403595
Epoch number : 207
Average Loss: 0.403536
Epoch number : 208
Average Loss: 0.403407
Epoch number : 209
Average Loss: 0.403024
Epoch number : 210
Average Loss: 0.402953
Epoch number : 211
Average Loss: 0.403022
Epoch number : 212
Average Loss: 0.402800
Epoch number : 213
Average Loss: 0.402473
Epoch number : 214
Average Loss: 0.402512
Epoch number : 215
Average Loss: 0.402809
Epoch number : 216
Average Loss: 0.402355
Epoch number : 217
Average Loss: 0.402385
Epoch number : 218
Average Loss: 0.402103
Epoch number : 219
Average Loss: 0

In [ ]:

from torchvision.utils import make_grid
import numpy as np
import torch

def sampler_pixel(model:minimalDiffRl_revised,nb_steps=128):
    tensor_temp = torch.ones(32,1,28,28).to(device=device)
    x_alpha = torch.randn_like(tensor_temp)
    for t in range(nb_steps):
        alpha_start = t/nb_steps
        alpha_end = (t+1)/nb_steps

        tensor_temp*= alpha_start

        x_alpha = x_alpha + (alpha_end - alpha_start)*model(x_alpha,tensor_temp)

    return x_alpha


time_steps = 200
samples = sampler_pixel(model,nb_steps=time_steps)


samples = samples.clamp(0.0, 1.0)
import matplotlib.pyplot as plt
%matplotlib inline

sample_grid = make_grid(samples, nrow=int(np.sqrt(32)))

plt.figure(figsize=(6,6))
plt.axis('off')
plt.imshow(sample_grid.permute(1,2,0).cpu(), vmin=0., vmax=1.)
plt.show()